In [1]:
import numpy as np
import pandas as pd

# Model A
from scipy import spatial
import operator

# Model B
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

In [2]:
users = pd.read_json("./IUM21Z_Zad_02_01/users.jsonl", lines=True)
sessions = pd.read_json("./IUM21Z_Zad_02_01/sessions.jsonl", lines=True)
products = pd.read_json("./IUM21Z_Zad_02_01/products.jsonl", lines=True)

# Model A

Sesje są grupowane po user_id oraz product_id.<br>
Każdej parze użytkownik-produkt jest przyporządkowany wynik, reprezentujący ile razy użytkownik wyświetlił dany produkt.<br>
Pary nie występujące w tabeli sessions są dodawane z wynikiem 0, ponieważ dane produkty nie były wyświetlone przez danego użytkownika.<br>
Do celów testowania dodawana jest pomocnicza kolumna user_view jednoznacznie określająca czy produkt zostal kiedykolwiek wyświetlony orzez użytkownika.

In [3]:
sessionsA = sessions.copy()
sessionsA['score'] = sessionsA['event_type'].map({'VIEW_PRODUCT':1, 'BUY_PRODUCT':0})

groupA = sessionsA.groupby(['user_id', 'product_id'])['score'].sum().reset_index()

groupA = pd.pivot_table(groupA, values='score', index='user_id', columns='product_id')
groupA = groupA.fillna(0)
groupA = groupA.stack().reset_index()
groupA = groupA.rename(columns={0:'score'})
groupA['user_view'] = groupA['score'].apply(lambda x: 1 if x > 0 else 0)
groupA

,user_id,product_id,score,user_view
0,102,1001,0.0,0
1,102,1002,3.0,1
2,102,1003,2.0,1
3,102,1004,2.0,1
4,102,1005,5.0,1
...,...,...,...,...
63795,301,1315,1.0,1
63796,301,1316,1.0,1
63797,301,1317,4.0,1
63798,301,1318,4.0,1


Zbiór danych jest dzielony na trenujący i testowy.<br>
Ta sama maska będzie później wykorzystana do podzielenia zbioru dla modelu B.

In [4]:
mask = np.random.rand(len(groupA)) < 0.8
trainsetA = groupA[mask]
testsetA = groupA[~mask]
trainsetA = trainsetA.reset_index()
trainsetA

,index,user_id,product_id,score,user_view
0,1,102,1002,3.0,1
1,2,102,1003,2.0,1
2,3,102,1004,2.0,1
3,4,102,1005,5.0,1
4,5,102,1006,5.0,1
...,...,...,...,...,...
51019,63792,301,1312,0.0,0
51020,63794,301,1314,0.0,0
51021,63796,301,1316,1.0,1
51022,63798,301,1318,4.0,1


In [5]:
def one_hot_encode(element, list):
    '''
    Funkcja koduje wskaźnikowo element, według pełnej listy możliwych elementów.
    '''
    one_hot_encode_list = []
    
    for e in list:
        if element == e:
            one_hot_encode_list.append(1)
        else:
            one_hot_encode_list.append(0)
    return one_hot_encode_list

Kolumna cateory_path jest zakodowana wskaźnikowo. Natomiast kolumny price i user_rating są normalizowane do przedziału [0, 1].

In [6]:
category_list = products['category_path'].unique()

productsA = products.copy()
productsA['category_path'] = productsA['category_path'].apply(lambda x: one_hot_encode(x, category_list))
productsA['price'] = (productsA['price'] - productsA['price'].min()) / (productsA['price'].max() - productsA['price'].min())
productsA['user_rating'] = (productsA['user_rating'] - productsA['user_rating'].min()) / (productsA['user_rating'].max() - productsA['user_rating'].min())
productsA

,product_id,product_name,category_path,price,user_rating
0,1001,Telefon Siemens Gigaset DA310,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.007590,0.932685
1,1002,Kyocera FS-1135MFP,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.268068,0.225297
2,1003,Kyocera FS-3640MFP,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1.000000,0.744302
3,1004,Fallout 3 (Xbox 360),"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.006414,0.662897
4,1005,Szalone Króliki Na żywo i w kolorze (Xbox 360),"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.006414,0.071347
...,...,...,...,...,...
314,1315,Jabra Talk,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",0.007069,0.073960
315,1316,Plantronics Voyager Legend,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",0.032469,0.732459
316,1317,Plantronics Savi W740,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",0.170589,0.818660
317,1318,Plantronics Savi W710,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0.072270,0.980461


In [7]:
def similarity(product_id1, product_id2):
    '''
    Wyznacza odległość między dwoma produktami.
    Odległość między kolumnami category_path jest wyznaczane jako cosinus kąta między dwoma wektorami.
    Odległość między kolumnami price i user_rating to odległość euklidesowa.
    '''
    a = productsA.iloc[product_id1]
    b = productsA.iloc[product_id2]
    
    categoryA = a['category_path']
    categoryB = b['category_path']
    category_distance = spatial.distance.cosine(categoryA, categoryB)
    
    priceA = a['price']
    priceB = b['price']
    price_distance = abs(priceA - priceB)
    
    ratingA = a['user_rating']
    ratingB = b['user_rating']
    rating_distance = abs(ratingA - ratingB)
    
    return category_distance + price_distance + rating_distance

In [8]:
def get_distances(product_id):
    '''
    Zwraca dystanse do wszystkich produktów od danego produktu.
    Produkty, których odległości są zwracane są w takiej samej kolejności jak w tabeli products.
    '''
    p = products.index[products['product_id'] == product_id][0]
    distances = []
    
    for index, product in products.iterrows():
        if product['product_id'] != product_id:
            dist = similarity(index, p)
            distances.append(dist)
        else:
            distances.append(0)
    
    return distances

In [9]:
def get_neighbours(distances, K):
    '''
    Zwraca indeksy K najbliższych sąsiadów na podstawie danych dystansów.
    '''
    distances = [(index, dist) for index, dist in enumerate(distances)]
    distances.sort(key=operator.itemgetter(1))
    neighbours = []
    
    for x in range(K):
        neighbours.append(distances[x])
    return neighbours

Obliczenie dystansu do każdego produktu dla każdego produktu.<br>
Wystarczy tą operację wykonać raz na samym początku. Dystanse mogą zostać zapisane do pliku i ładowane przy starcie systemu, aby nie wyonywać zbędnych obliczeń.<br>
Po każdym dodaniu nowego produktu należy uaktualnić dystanse, tak by zawierały nowe produkty. 

In [10]:
all_distances = []
for i in range(len(products)):
    all_distances.append(get_distances(products['product_id'].iloc[i]))

In [11]:
def get_recommendationA(user_id, dataset, K, all_distances):
    '''
    Zwraca indeksy K "najlepszych" rekomendacji dla podanego użytkownika.
    Jako dataset należy podać zbiór danych zawierający liczbę "kliknięć" dla każdej pary użytkownik-produkt.
    Rekomendacje nie zawierają produktów, które użytkownik już kiedyś wyświetlił.
    '''
    s = dataset.index[dataset['user_id'] == user_id].tolist()
    ids = []
    for i in s:
        ids.append(dataset['product_id'].iloc[i])
    
    distances = [0] * len(products)
    for id in ids:
        i = products.index[products['product_id'] == id][0]
        score = dataset.loc[(dataset['user_id'] == user_id) & (dataset['product_id'] == id), 'score']
        if len(score) != 0:
            score = score.item()
            for p in range(len(products)):
                distances[p] = distances[p] + all_distances[i][p] * score
    
    maxi = max(distances)
    for id in ids:
        distances[products.index[products['product_id'] == id][0]] = maxi
    
    return get_neighbours(distances, K)

Losowo wybierani użytkownicy, dla których zostaną przygotowane rekomendacje.<br>
Dla tych samych użytkowników później zostaną przygotowane rekomendacje modelu B.

In [12]:
user_mask = np.random.rand(len(users)) < 0.05
test_users = users[user_mask]
test_users

,user_id,name,city,street
1,103,Daniel Prawdzik,Wrocław,ulica Okrężna 35
3,105,Wojciech Foremniak,Poznań,ulica Jeżynowa 92/93
47,149,Melania Dubel,Gdynia,plac Mała 14/36
81,183,Kacper Lepak,Szczecin,ulica Bociania 75
83,185,Konrad Piotrowiak,Poznań,ul. Targowa 16/32
102,204,Kamil Smektała,Radom,ul. Górnicza 87/09
128,230,Kacper Majtczak,Szczecin,ul. Chopina 00
141,243,Jeremi Klucznik,Wrocław,al. Wojciecha 96/11
142,244,Justyna Babiuch,Wrocław,plac Wilcza 29/83
181,283,Maksymilian Juroszek,Gdynia,aleja Osiedlowa 148


Wyliczanie współczynnika trafnych rekomendacji do wszystkich rekomendacji, dla użytkowników w tabeli test_users.<br>
Dla każdego użytkownika jest generowane K rekomendacji, których trafność jest oceniana.

In [13]:
K = 5
correct = 0
for index, user in test_users.iterrows():
    print(user['user_id'])
    recommendations = get_recommendationA(user['user_id'], trainsetA, K, all_distances)
    for recommendation in recommendations:
        id = products.iloc[recommendation[0]]['product_id']
        view = testsetA[(testsetA['product_id'] == id) & (testsetA['user_id'] == user['user_id'])]['user_view']
        if len(view) != 0:
            view = view.item()
            if view == 1:
                correct = correct + 1
correct / (K * len(test_users))

103
105
149
183
185
204
230
243
244
283
292
299


0.4

# Model B

Sesje są grupowane po user_id oraz product_id tak samo jak w poprzednim modelu.<br>
Każdej parze użytkownik-produkt jest przyporządkowany wynik, reprezentujący ile razy użytkownik wyświetlił dany produkt.<br>
Pary nie występujące w tabeli sessions są dodawane z wynikiem 0, ponieważ dane produkty nie były wyświetlone przez danego użytkownika.<br>
Do celów testowania dodawana jest pomocnicza kolumna user_view jednoznacznie określająca czy produkt zostal kiedykolwiek wyświetlony orzez użytkownika oraz kolumna interaction_score, która jest normalizacją kolumny score do przedziału [0, 1]. Zadaniem modelu będzie przewidzenie wartości interaction_score.

In [14]:
sessionsB = sessions.copy()
sessionsB['score'] = sessionsB['event_type'].map({'VIEW_PRODUCT':5, 'BUY_PRODUCT':5})

groupB = sessionsB.groupby(['user_id', 'product_id'])['score'].sum().reset_index()

groupB = sessionsB.groupby(['user_id', 'product_id'])['score'].sum().reset_index()
groupB['score'] = groupB['score'].apply(lambda x: 5 if x>5 else x)
groupB = pd.pivot_table(groupB, values='score', index='user_id', columns='product_id')
groupB = groupB.fillna(0)
groupB = groupB.stack().reset_index()
groupB = groupB.rename(columns={0:'score'})
groupB['user_view'] = groupB['score'].apply(lambda x: 1 if x > 0 else 0)

std1 = MinMaxScaler(feature_range=(0, 1))
std1.fit(groupB['score'].values.reshape(-1,1))
groupB['interaction_score'] = std1.transform(groupB['score'].values.reshape(-1,1))
groupB

,user_id,product_id,score,user_view,interaction_score
0,102,1001,0.0,0,0.0
1,102,1002,5.0,1,1.0
2,102,1003,5.0,1,1.0
3,102,1004,5.0,1,1.0
4,102,1005,5.0,1,1.0
...,...,...,...,...,...
63795,301,1315,5.0,1,1.0
63796,301,1316,5.0,1,1.0
63797,301,1317,5.0,1,1.0
63798,301,1318,5.0,1,1.0


In [15]:
def price_bin(price):
    '''
    Przyporządkowuje cenie odpowiednią kategorię.
    '''
    if price <= 25:
        return 0
    if price <= 50:
        return 1
    if price <= 100:
        return 2
    if price <= 250:
        return 3
    if price <= 500:
        return 4
    if price <= 1000:
        return 5
    if price <= 2000:
        return 6
    if price <= 4000:
        return 7
    else:
        return 8

In [16]:
def rating_bin(rating):
    '''
    Przyporządkowuje ocenie odpowiednią kategorię.
    '''
    if rating <= 0.5:
        return 0
    if rating <= 1.5:
        return 1
    if rating <= 2.5:
        return 2
    if rating <= 3.5:
        return 3
    if rating <= 4.5:
        return 4
    else:
        return 5

Dodawanie dodatkowych informacji o produktach takich jak: kategoria, cena i ocena.<br>
Cena i ocena produkty są kategoryzowane przez powyższe funkcje.

In [17]:
groupB = pd.merge(groupB, products, on="product_id", how="left")
groupB = groupB[['user_id', 'product_id', 'product_name', 'category_path', 'price', 'user_rating', 'score', 'interaction_score', 'user_view']]
groupB['price'] = groupB['price'].apply(lambda x: price_bin(x))
groupB['user_rating'] = groupB['user_rating'].apply(lambda x: rating_bin(x))
groupB

,user_id,product_id,product_name,category_path,price,user_rating,score,interaction_score,user_view
0,102,1001,Telefon Siemens Gigaset DA310,Telefony i akcesoria;Telefony stacjonarne,2,5,0.0,0.0,0
1,102,1002,Kyocera FS-1135MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,7,1,5.0,1.0,1
2,102,1003,Kyocera FS-3640MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,8,4,5.0,1.0,1
3,102,1004,Fallout 3 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,1,3,5.0,1.0,1
4,102,1005,Szalone Króliki Na żywo i w kolorze (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,1,0,5.0,1.0,1
...,...,...,...,...,...,...,...,...,...
63795,301,1315,Jabra Talk,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,2,0,5.0,1.0,1
63796,301,1316,Plantronics Voyager Legend,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,3,4,5.0,1.0,1
63797,301,1317,Plantronics Savi W740,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,6,4,5.0,1.0,1
63798,301,1318,Plantronics Savi W710,Sprzęt RTV;Audio;Słuchawki,5,5,5.0,1.0,1


Dzielenie zbioru na trenujący i testowy według tej samej maski co w modelu A.

In [18]:
trainsetB = groupB[mask]
testsetB = groupB[~mask]
trainsetB = trainsetB.reset_index()
trainsetB

,index,user_id,product_id,product_name,category_path,price,user_rating,score,interaction_score,user_view
0,1,102,1002,Kyocera FS-1135MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,7,1,5.0,1.0,1
1,2,102,1003,Kyocera FS-3640MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,8,4,5.0,1.0,1
2,3,102,1004,Fallout 3 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,1,3,5.0,1.0,1
3,4,102,1005,Szalone Króliki Na żywo i w kolorze (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,1,0,5.0,1.0,1
4,5,102,1006,Call of Duty 4 Modern Warfare (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,2,2,5.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...
51019,63792,301,1312,One For All SV 9335,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...,2,3,0.0,0.0,0
51020,63794,301,1314,Assassin&#39;s Creed (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,1,2,0.0,0.0,0
51021,63796,301,1316,Plantronics Voyager Legend,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,3,4,5.0,1.0,1
51022,63798,301,1318,Plantronics Savi W710,Sprzęt RTV;Audio;Słuchawki,5,5,5.0,1.0,1


Przekształcenie tabeli zawierąjych pary użytkownik-produkt na macierz, której wartości są kolumną score.

In [19]:
train_matrix = pd.pivot_table(trainsetB, values='score', index='user_id', columns='product_id')
train_matrix = train_matrix.fillna(0)
train_matrix

product_id,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319
user_id,,,,,,,,,,,,,,,,,,,,,
102,0.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0.0,5.0,...,0.0,5.0,0.0,0.0,5.0,5.0,0.0,5.0,0.0,0.0
103,0.0,5.0,0.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,0.0,0.0,0.0,0.0,5.0,5.0,5.0,0.0,5.0,5.0
104,0.0,5.0,5.0,5.0,0.0,5.0,0.0,5.0,0.0,5.0,...,0.0,5.0,5.0,0.0,5.0,5.0,0.0,5.0,0.0,0.0
105,5.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,5.0,...,0.0,5.0,0.0,5.0,5.0,0.0,5.0,0.0,0.0,5.0
106,0.0,0.0,0.0,5.0,5.0,0.0,5.0,5.0,5.0,5.0,...,5.0,5.0,0.0,5.0,0.0,0.0,0.0,5.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,0.0,5.0,0.0,5.0,5.0,5.0,0.0,5.0,5.0,5.0,...,0.0,0.0,0.0,5.0,0.0,5.0,5.0,5.0,0.0,5.0
298,0.0,5.0,5.0,5.0,5.0,5.0,0.0,5.0,5.0,0.0,...,0.0,5.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0
299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Tabela zawierająca wszystkie wykorzystywane kategorie dla każdego produktu.

In [20]:
product_cat = trainsetB[['product_id', 'category_path', 'price', 'user_rating']].drop_duplicates('product_id')
product_cat = product_cat.sort_values(by='product_id')
product_cat

,product_id,category_path,price,user_rating
241,1001,Telefony i akcesoria;Telefony stacjonarne,2,5
0,1002,Komputery;Drukarki i skanery;Biurowe urządzeni...,7,1
1,1003,Komputery;Drukarki i skanery;Biurowe urządzeni...,8,4
2,1004,Gry i konsole;Gry na konsole;Gry Xbox 360,1,3
3,1005,Gry i konsole;Gry na konsole;Gry Xbox 360,1,0
...,...,...,...,...
237,1315,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,2,0
498,1316,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,3,4
238,1317,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,6,4
239,1318,Sprzęt RTV;Audio;Słuchawki,5,5


Dla każdej wykorzystywanej kategorii jest wyliczana macierz podobieństw między produktami.<br>
W przypadku ceny i oceny produktu liczona jest odległość euklidesowa.
W przypadku ścieżki kategorii produktu kolumna jest najpierw zakodowana wskaźnikowo, w następujący sposób:
Kolumn jest tyle ile różnych słów we wszystkich kategoriach, jeśli dane słowo występuje w ścieżce kategorii na odpowiadającym mu miejscu znajduje się jedynka. Przykładowe słowa: "360", "dvd", "mp3", "słuchawki", "xbox".<br>
Następnie wszystkie macierze są składane w jedną macierz podobieństw.
Na podstawie macierzy podobieńst wyliczane są przewidywane wartości interaction_score znormalizowane do przedziału [0, 1].

In [21]:
price_matrix = np.reciprocal(euclidean_distances(np.array(product_cat['price']).reshape(-1,1))+1)
euclidean_matrix1 = pd.DataFrame(price_matrix,columns=product_cat['product_id'],index=product_cat['product_id'])

rating_matrix = np.reciprocal(euclidean_distances(np.array(product_cat['user_rating']).reshape(-1,1))+1)
euclidean_matrix2 = pd.DataFrame(rating_matrix,columns=product_cat['product_id'],index=product_cat['product_id'])

tfidf_vectorizer = TfidfVectorizer()
doc_term = tfidf_vectorizer.fit_transform(list(product_cat['category_path']))
dt_matrix = pd.DataFrame(doc_term.toarray().round(3), index=[i for i in product_cat['product_id']], columns=tfidf_vectorizer.get_feature_names())
cos_similar_matrix = pd.DataFrame(cosine_similarity(dt_matrix.values),columns=product_cat['product_id'],index=product_cat['product_id'])

similarity_matrix = euclidean_matrix1.multiply(euclidean_matrix2).multiply(cos_similar_matrix)
content_matrix = train_matrix.dot(similarity_matrix)
std2 = MinMaxScaler(feature_range=(0, 1))
std2.fit(content_matrix.values)
content_matrix = std2.transform(content_matrix.values)
content_matrix = pd.DataFrame(content_matrix,columns=sorted(trainsetB['product_id'].unique()),index=sorted(trainsetB['user_id'].unique()))
content_matrix

,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319
102,0.374331,0.787404,0.651193,0.939232,0.808453,0.916697,1.000000,0.937406,0.864525,0.916697,...,0.672053,0.922709,0.752868,0.823518,0.926142,0.723509,0.553152,0.689100,0.409053,0.385867
103,0.647366,0.842672,0.504909,0.885402,0.907531,0.888388,0.920607,0.888285,0.826355,0.888388,...,0.885799,0.829533,0.955835,0.861467,0.885972,0.961015,0.987311,0.645291,0.942192,0.950318
104,0.367859,0.867349,0.932926,0.746317,0.626206,0.702093,0.648630,0.705019,0.603897,0.702093,...,0.706254,0.683486,0.789273,0.718446,0.712366,0.661928,0.478236,0.682886,0.360309,0.361427
105,0.973862,0.183710,0.316690,0.633681,0.547290,0.611759,0.700133,0.667355,0.613425,0.611759,...,0.557349,0.748577,0.694811,0.885461,0.575380,0.599745,0.909639,0.503931,0.432341,0.958898
106,0.455133,0.438463,0.624931,0.850780,0.771313,0.811835,0.869039,0.810448,0.807463,0.811835,...,0.697225,0.816361,0.663055,0.854170,0.792684,0.493966,0.532491,0.682772,0.960185,0.410638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,0.518684,0.848662,0.480840,0.801013,0.736317,0.802113,0.765436,0.757194,0.754908,0.802113,...,0.523489,0.652123,0.727674,0.852955,0.733882,0.877834,0.849683,0.830077,0.401203,0.906278
298,0.398266,0.795948,0.677754,0.910754,0.864182,0.962530,0.899239,0.858951,0.899858,0.962530,...,0.653905,0.775202,0.672096,0.632025,0.932398,0.703694,0.556068,0.401409,0.344581,0.294105
299,0.079193,0.073348,0.033144,0.185747,0.181855,0.151860,0.129008,0.157894,0.153137,0.151860,...,0.093935,0.098428,0.130703,0.237600,0.173057,0.061983,0.091985,0.095506,0.115704,0.090384
300,0.008791,0.017708,0.005491,0.147124,0.107313,0.138661,0.158074,0.119330,0.109965,0.138661,...,0.062752,0.149071,0.076330,0.040521,0.118953,0.021862,0.015545,0.009220,0.022453,0.015865


Macierz zawierająca predykcje interaction_score dla par użytkownik-produkt jest przekonwertowana na tabelę dla łatwiejszego dostępu.<br>
Jest to odpowiednik listy all_distances z modelu A. Wystarczy policzyć raz na początku startu systemu i można na jej podstawie generować rekomendacje.<br>
Tym razem jednak aktualizacje będą potrzebne nie tylko przy dojściu nowych produktów, ale także nowych klientów.

In [22]:
content_df = content_matrix.stack().reset_index()
content_df = content_df.rename(columns={'level_0':'user_id','level_1':'product_id',0:'predicted_interaction'})
content_df

,user_id,product_id,predicted_interaction
0,102,1001,0.374331
1,102,1002,0.787404
2,102,1003,0.651193
3,102,1004,0.939232
4,102,1005,0.808453
...,...,...,...
63795,301,1315,0.337611
63796,301,1316,0.568956
63797,301,1317,0.345820
63798,301,1318,0.753735


In [23]:
def get_recommendationB(content_df, user_id, dataset, K):
    '''
    Zwraca id produktów K "najlepszych" rekomendacji dla podanego użytkownika.
    Jako dataset należy podać zbiór danych zawierający liczbę "kliknięć" dla każdej pary użytkownik-produkt.
    Rekomendacje nie zawierają produktów, które użytkownik już kiedyś wyświetlił.
    '''
    s = dataset.index[dataset['user_id'] == user_id].tolist()
    ids = []
    for i in s:
        ids.append(dataset['product_id'].iloc[i])
        
    user_content_df = content_df.loc[content_df['user_id'] == user_id]
    
    mini = user_content_df['predicted_interaction'].min()
    for id in ids:
        user_content_df.loc[user_content_df['product_id'] == id, 'predicted_interaction'] = mini
        
    user_content_df = user_content_df.sort_values(by="predicted_interaction", ascending=False)
    
    recommendations = []
    for x in range(K):
        recommendations.append(user_content_df['product_id'].iloc[x])
    return recommendations

Wyliczanie współczynnika trafnych rekomendacji do wszystkich rekomendacji, dla użytkowników w tabeli test_users.<br>
Dla każdego użytkownika jest generowane K rekomendacji, których trafność jest oceniana.

In [24]:
K = 5
correct = 0
for index, user in test_users.iterrows():
    print(user['user_id'])
    recommendations = get_recommendationB(content_df, user['user_id'], trainsetB, K)
    for recommendation in recommendations:
        view = testsetB[(testsetB['product_id'] == recommendation) & (testsetB['user_id'] == user['user_id'])]['user_view']
        if len(view) != 0:
            view = view.item()
            if view == 1:
                correct = correct + 1
correct / (K * len(test_users))

103
105


C:\Users\Jakub\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


149
183
185
204
230
243
244
283
292
299


0.43333333333333335

# Wnioski

Przy współczynniku alfa = 50/363 = 0.14 oba modele spełniły analityczne kryterium sukcesu.<br>
Natomiast model B osiągnął nie tylko lepszy wynik, ale także zajmuje mniej czasu do policzenia rekomendacji. Dodatkowo policzenie listy all_distances jest wielokrotnie bardziej czasochłonne niż policzenie tabeli content_df.
<br><br>
Jedyną wadą modelu B, jest fakt że trzeba go aktualizować za każdym razem jak do bazy danych zostanie dodany nowy użytkownik lub produkt, a nie tylko w przypadku produktu jak w modelu A.
<br><br>
Obie metody opierają się na podobnych metodach, liczą odległość między produktami. Jednak model B jest bardziej rozbudowany.<br>
Model A operuje tylko i wyłącznie na tych odległościach i wybiera produkty najbardziej zbliżone do listy produktów wyświetlonych przez klienta.<br>
Model B dodatkowo daje każdej parze użytkownik-produkt wynik, na którego podstawie ocenia czy klient wyświetli dany produkt czy nie. "Najlepsze" rekomendacje w jego przypadku są wybierane jako produkty z najwyższym wynikiem dla danego klienta.

# Widoki mikroserwisu

url: http://127.0.0.1:8000/api/recommendations/simple/1002/<br>
Dla zadanego id produktu zwraca podobne produkty.

![title](imgs/screen1.png)

url: http://127.0.0.1:8000/api/recommendations/advanced/102/<br>
Dla zadanego id użytkownika zwraca rekomendowane produkty (model A).

![title](imgs/screen2.png)

url: http://127.0.0.1:8000/api/recommendations/best/102/<br>
Dla zadanego id użytkownika zwraca rekomendowane produkty (model B).

![title](imgs/screen3.png)

url: http://127.0.0.1:8000/api/recommendations/ab/testset/<br>
Zwraca dokładność rekomendacji dla obydwu modeli na zbiorze testowym.

![title](imgs/screen4.png)

url: http://127.0.0.1:8000/api/recommendations/ab/?id=103,105,149,183,185,204,230,243,244,283,292,299<br>
id - parametr<br>
Dla zadanych użytkowników wykonuje test A/B; zwraca dokładność rekomendacji dla danego modelu oraz rekomendowane produkty.

![title](imgs/screen5.png)